# Perturbation Double Sweep
TJ Kim

1.6.22 

#### Summary:
- Sweep both Linf norm [0,1,0.5,1.0,2.0,4.0] for defense and attack
- Generate tables for adv_acc, adv_target, adv_miss

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


#### Import Relevant Libraries

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

#### Function - Calculate Mean without Diagonal

In [3]:
# Perform np.mean without the diagonal
def avg_nondiag(array2d):
    d1 = array2d.shape[0]
    d2 = array2d.shape[1]
    
    counter = 0
    val = 0
    
    for i1 in range(d1):
        for i2 in range(d2):
            if i1 != i2:
                counter+=1
                val += array2d[i1,i2]
    
    return val/counter

#### Generate Dummy Aggregator and Client

In [4]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = "FedEM_adv"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 3
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/21_12_30_feddef2_n40_linf0_5/'
args_.validation = False
args_.num_user = 40

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 208.20it/s]


===> Initializing clients..


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:47<00:00,  1.69it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.292 | Train Acc: 12.195% |Test Loss: 2.292 | Test Acc: 12.291% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


#### Load Dataset From Client Validation

In [5]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.stack(data_y)

# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)

#### Import Experiment Name Information

Used later to loop through.

In [6]:
# exp_names = ['21_12_30_feddef_n40_linf0_5_G0_0/','21_12_30_feddef_n40_linf0_5_G0_25/', '21_12_30_feddef_n40_linf0_5/',
#              '21_12_30_feddef_n40_linf0_5_G0_75/', '21_12_30_feddef_n40_linf0_5_G1_0/']

# R - Sweep Resources
# exp_names = ['22_01_01_feddef_n40_linf0_5_G0_5_R0_1/','22_01_01_feddef_n40_linf0_5_G0_5_R0_2/',
#              '22_01_01_feddef_n40_linf0_5_G0_5_R0_4/', '21_12_30_feddef_n40_linf0_5/', 
#              '22_01_01_feddef_n40_linf0_5_G0_5_R0_6/']

# G - Full Resources
# exp_names = ['21_12_30_feddef_n40_linf0_5_G0_0/','22_01_02_feddef_n40_linf0_5_G0_25_R1_0/',
#              '22_01_02_feddef_n40_linf0_5_G0_5_R1_0/', '22_01_02_feddef_n40_linf0_5_G0_75_R1_0/',
#              '22_01_02_feddef_n40_linf0_5_G1_0_R1_0/']

# Q - adv freq
# exp_names = ['22_01_03_feddef_n40_linf0_5_G0_5_R1_0_Q3/', '22_01_03_feddef_n40_linf0_5_G0_5_R1_0_Q5/',
#              '22_01_02_feddef_n40_linf0_5_G0_5_R1_0/', '22_01_03_feddef_n40_linf0_5_G0_5_R1_0_Q20/']

# Ep - perturbation amount
exp_names = ['22_01_04_feddef_n40_linf0_5_G0_5_R1_0_Q10_eps0_1/', '22_01_02_feddef_n40_linf0_5_G0_5_R1_0/',
             '22_01_04_feddef_n40_linf0_5_G0_5_R1_0_Q10_eps1/', '22_01_04_feddef_n40_linf0_5_G0_5_R1_0_Q10_eps2/',
             '22_01_04_feddef_n40_linf0_5_G0_5_R1_0_Q10_eps4/']

# exp_names = ['22_01_06_feddef_n40_linf0_5_G0_5_R1_0_Q10_eps0_05/', '22_01_04_feddef_n40_linf0_5_G0_5_R1_0_Q10_eps0_1/',
#              '22_01_06_feddef_n40_linf0_5_G0_5_R1_0_Q10_eps0_15/', '22_01_06_feddef_n40_linf0_5_G0_5_R1_0_Q10_eps0_2/',
#              '22_01_06_feddef_n40_linf0_5_G0_5_R1_0_Q10_eps0_3/']

# Benign Model Run
# exp_names = ['21_12_30_feddef_n40_linf0_5_G0_0/']

base = 'weights/cifar/'
train_item = 'train_client_weights.npy'

# Attack Perturbation Amount
atk_eps = [0.1, 0.5, 1.0, 2.0, 4.0, 8.0, 16.0]

#### Set Measurement Metrics

In [7]:
num_victims = 8
num_exp = len(exp_names)
# Set Up Dictionaries -- list holds the adversary idx
exp_logs = {}

for j in range(num_exp):
    logs_adv = []

    for i in range(num_victims):
        adv_dict = {}
        adv_dict['orig_acc_transfers'] = None
        adv_dict['orig_similarities'] = None
        adv_dict['adv_acc_transfers'] = None
        adv_dict['adv_similarities_target'] = None
        adv_dict['adv_similarities_untarget'] = None
        adv_dict['adv_target'] = None
        adv_dict['adv_miss'] = None
        adv_dict['metric_alignment'] = None
        adv_dict['ib_distance_legit'] = None
        adv_dict['ib_distance_adv'] = None
        
        logs_adv += [adv_dict]
    
    exp_logs[j] = logs_adv


In [8]:
# Generate extra set of exp_logs one for each perturbation
exp_logs_list = []

for i in range(len(atk_eps)):
    exp_logs_list += [copy.deepcopy(exp_logs)]

#### Load Each Model and Perform Metric

In [9]:
# Inter Boundary Distance Metric
num_trials = 50
batch_size = 5000


for j in range(num_exp):
    print('processing file', exp_names[j], '...')
    
    # Change name if need be
    args_.save_path = base + exp_names[j]

    # Import weights for aggregator
    aggregator.load_state(args_.save_path)

    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]
        
    weight_name = args_.save_path + train_item
    weights = np.load(weight_name)
    np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

    # Set model weights
    model_weights = []
    num_models = num_victims

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]
    
    for ep_idx in range(len(atk_eps)):
        # Run Measurements for both targetted and untargeted analysis
        victim_idxs = range(num_victims)

        t1 = Transferer(models_list=models_test, dataloader=dataloader)
        t1.generate_victims(victim_idxs)

        t2 = Boundary_Transferer(models_list=models_test, dataloader=dataloader)
        t2.victim_idx = victim_idxs

        for adv_idx in victim_idxs:
            print("\t Adv idx:", adv_idx)
            # Perform Attacks
            t1.atk_params = PGD_Params()
            t1.atk_params.set_params(batch_size=500, iteration = 10,
                           target = 5, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                           step_size = 0.05, step_norm = "inf", eps = atk_eps[ep_idx], eps_norm = 2)

            t1.generate_advNN(adv_idx)
            t1.generate_xadv(atk_type = "pgd")
            t1.send_to_victims(victim_idxs)
            # t1.check_empirical_metrics(orig_flag = True)

            # Log Performance
            exp_logs_list[ep_idx][j][adv_idx]['orig_acc_transfers'] = copy.deepcopy(t1.orig_acc_transfers)
            exp_logs_list[ep_idx][j][adv_idx]['orig_similarities'] = copy.deepcopy(t1.orig_similarities)
            exp_logs_list[ep_idx][j][adv_idx]['adv_acc_transfers'] = copy.deepcopy(t1.adv_acc_transfers)
            exp_logs_list[ep_idx][j][adv_idx]['adv_similarities_target'] = copy.deepcopy(t1.adv_similarities)        
            exp_logs_list[ep_idx][j][adv_idx]['adv_target'] = copy.deepcopy(t1.adv_target_hit)
            # exp_logs_list[ep_idx][j][adv_idx]['metric_alignment'] = copy.deepcopy(t1.metric_alignment)

            # Miss attack
            t1.atk_params.set_params(batch_size=500, iteration = 10,
                           target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                           step_size = 0.05, step_norm = "inf", eps = atk_eps[ep_idx], eps_norm = 2)
            t1.generate_xadv(atk_type = "pgd")
            t1.send_to_victims(victim_idxs)
            exp_logs_list[ep_idx][j][adv_idx]['adv_miss'] = copy.deepcopy(t1.adv_acc_transfers)
            exp_logs_list[ep_idx][j][adv_idx]['adv_similarities_untarget'] = copy.deepcopy(t1.adv_similarities)
        
        del t1

    del models_test

processing file 22_01_04_feddef_n40_linf0_5_G0_5_R1_0_Q10_eps0_1/ ...
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
processing file 22_01_02_feddef_n40_linf0_5_G0_5_R1_0/ ...
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2


#### Obtain Metric Values and Make Table

In [10]:
# Make a pandas table across double sweep from list
def make_metric_table(exp_list, metric, row_names, col_names, avg_diag_flag = True):
    
    num_col1 = len(exp_list)
    num_col2 = len(exp_list[0])
    num_victims = len(exp_list[0][0])
    victim_idxs = range(num_victims)
    exp_values = {}
    
    final_table = np.zeros([num_col1, num_col2])
    
    for j in range(num_col1): # Attack perturbation amount
        for k in range(num_col2): # Defense perturbation amount (Experiment)
            orig_vals = np.zeros([num_victims, num_victims])
            
            for adv_idx in range(num_victims):
                for victim in range(num_victims):
                    curr_list = exp_list[j][k]
                    orig_vals[adv_idx,victim] = curr_list[victim_idxs[adv_idx]][metric][victim_idxs[victim]].data.tolist()
            
            if avg_diag_flag:
                final_table[j,k] = avg_nondiag(orig_vals)
            else:
                final_table[j,k] = np.mean(orig_vals)
    
    df = pd.DataFrame(final_table, columns = col_names, index = row_names)
    
    return df

In [33]:
metrics = ['orig_acc_transfers','orig_similarities','adv_acc_transfers','adv_similarities_target',
           'adv_similarities_untarget','adv_target','adv_miss'] #,'metric_alignment']

In [36]:
row_names = ['A ep0.1', 'A ep0.5', 'A ep1.0', 'A ep2.0','A ep4.0', 'A ep8.0', 'A ep16.0']
col_names = ['D ep0.1', 'D ep0.1', 'D ep0.15', 'D ep0.2','D ep0.3']
# col_names = ['benign']

name = 'adv_target'
print(name)
make_metric_table(exp_logs_list, name, row_names, col_names, avg_diag_flag = True).T

adv_target


,A ep0.1,A ep0.5,A ep1.0,A ep2.0,A ep4.0,A ep8.0,A ep16.0
benign,0.03477,0.04722,0.081712,0.178227,0.411275,0.65046,0.675992


In [37]:
name = 'adv_miss'
print(name)
make_metric_table(exp_logs_list, name, row_names, col_names, avg_diag_flag = True).T

adv_miss


,A ep0.1,A ep0.5,A ep1.0,A ep2.0,A ep4.0,A ep8.0,A ep16.0
benign,0.822214,0.771071,0.673393,0.406607,0.156393,0.060143,0.055


In [38]:
name = 'orig_acc_transfers'
print(name)
make_metric_table(exp_logs_list, name, row_names, col_names, avg_diag_flag = True).T

orig_acc_transfers


,A ep0.1,A ep0.5,A ep1.0,A ep2.0,A ep4.0,A ep8.0,A ep16.0
benign,0.833607,0.824929,0.829464,0.830143,0.832107,0.832821,0.828107


In [39]:
name = 'adv_acc_transfers'
print(name)
make_metric_table(exp_logs_list, name, row_names, col_names, avg_diag_flag = True).T

adv_acc_transfers


,A ep0.1,A ep0.5,A ep1.0,A ep2.0,A ep4.0,A ep8.0,A ep16.0
benign,0.835426,0.815645,0.785748,0.672763,0.430575,0.21219,0.174202
